In [1]:
import pandas as pd

df = pd.read_csv('./covid/covid-monthly-cases.csv')

states = sorted(list(df.state.unique()))
years = sorted(list(df.submission_year.unique()))
months = sorted(list(df.submission_month.unique()))

df

,state,submission_year,submission_month,new_case,new_death,population,pnew_case,pnew_death
0,AK,2020,1,0.0,0.0,733391,0.000000,0.000000
1,AK,2020,2,0.0,0.0,733391,0.000000,0.000000
2,AK,2020,3,128.0,6.0,733391,0.000175,0.000008
3,AK,2020,4,227.0,3.0,733391,0.000310,0.000004
4,AK,2020,5,108.0,4.0,733391,0.000147,0.000005
...,...,...,...,...,...,...,...,...
1219,WY,2021,8,9897.0,82.0,576851,0.017157,0.000142
1220,WY,2021,9,15466.0,138.0,576851,0.026811,0.000239
1221,WY,2021,10,12324.0,178.0,576851,0.021364,0.000309
1222,WY,2021,11,8163.0,254.0,576851,0.014151,0.000440


In [30]:
import numpy as np

def is_zero(v, zero=0.00001):
    if np.abs(v) < zero:
        return 0.0
    return v

def adjust(df):
    for r, s in enumerate(df.sum(axis=1)):
        if s == 0:
            df.iloc[r,:] = 1
    return df

def normalize(df):
    return pd.DataFrame([df.iloc[r,:] * s
                         for r, s in enumerate(1 / df.sum(axis=1))],
                        index=df.index, columns=df.columns)

yr = 2020
mo = 10
field = 'pnew_case'
q = (df.submission_year==yr) & (df.submission_month==mo)

stats = {r.state: r[field] for _, r in df[q][['state', field]].iterrows()}
M = pd.DataFrame([[stats[states[i]] / stats[states[j]] for j in range(len(states))] 
                  for i in range(len(states))], columns=states, index=states)
# M = normalize(adjust(M))
M = adjust(M)
M.head()

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
AK,1.000000,1.349903,1.081880,2.662407,3.633631,1.524842,2.702679,3.638243,2.313498,2.240454,...,0.384525,1.101415,1.967525,0.804737,2.623873,15.899866,3.731789,0.554757,2.131967,0.803354
AL,0.740794,1.000000,0.801450,1.972295,2.691773,1.129594,2.002129,2.695190,1.713826,1.659715,...,0.284854,0.815922,1.457531,0.596144,1.943750,11.778529,2.764488,0.410961,1.579349,0.595120
AR,0.924317,1.247738,1.000000,2.460908,3.358627,1.409437,2.498133,3.362891,2.138406,2.070890,...,0.355423,1.018056,1.818617,0.743832,2.425291,14.696519,3.449356,0.512772,1.970614,0.742554
AZ,0.375600,0.507023,0.406354,1.000000,1.364792,0.572731,1.015126,1.366524,0.868950,0.841515,...,0.144427,0.413691,0.739002,0.302259,0.985527,5.971990,1.401660,0.208367,0.800767,0.301740
CA,0.275207,0.371502,0.297741,0.732713,1.000000,0.419647,0.743796,1.001269,0.636690,0.616588,...,0.105824,0.303117,0.541476,0.221469,0.722108,4.375752,1.027014,0.152673,0.586732,0.221089


In [39]:
M.sum(axis=1)

AK    124.334359
AL     92.106175
AR    114.924380
AZ     46.699989
CA     34.217666
CO     81.539193
CT     46.004112
DC     34.174285
DE     53.743018
FL     55.495159
GA     60.412262
HI     20.992030
IA    149.003283
ID    149.002191
IL    114.519310
IN    106.222592
KS    105.149396
KY     97.522648
LA     50.047351
MA     45.719341
MD     40.576207
ME     11.029059
MI     80.162102
MN    105.020116
MO    129.511316
MS     87.386058
MT    217.431112
NC     74.160062
ND    355.861698
NE    160.740613
NH     24.848522
NJ     51.446888
NM     97.832716
NV     80.902369
NY     17.835988
OH     63.526390
OK    121.643874
OR     32.714281
PA     46.332121
RI     95.922913
SC     67.935975
SD    323.345619
TN    112.886071
TX     63.193296
UT    154.503193
VA     47.385814
VT      7.819837
WA     33.317629
WI    224.123942
WV     58.319079
WY    154.769012
dtype: float64

In [35]:
(M.loc['AK'] / M.loc['AK'].sum())[0:5]

AK    0.008043
AL    0.010857
AR    0.008701
AZ    0.021413
CA    0.029225
Name: AK, dtype: float64

In [36]:
(M.loc['AL'] / M.loc['AL'].sum())[0:5]

AK    0.008043
AL    0.010857
AR    0.008701
AZ    0.021413
CA    0.029225
Name: AL, dtype: float64

In [38]:
(M.loc['CA'] / M.loc['CA'].sum())[0:5]

AK    0.008043
AL    0.010857
AR    0.008701
AZ    0.021413
CA    0.029225
Name: CA, dtype: float64